# Module

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# from lightgbm import LGBMRegressor
import xgboost as xgb

In [7]:
df = pd.read_csv('data/back_hospital.csv')
df_back = df.drop(['Unnamed: 0'], axis=1)
df_back

,back_hospital.yyyymmdd,back_hospital.area,back_hospital.sex,back_hospital.frequency
0,20120101,강원,1,3
1,20120101,강원,2,3
2,20120101,경기,1,4
3,20120101,경기,2,5
4,20120101,경남,1,2
...,...,...,...,...
49669,20151231,제주,2,0
49670,20151231,충남,1,0
49671,20151231,충남,2,0
49672,20151231,충북,1,1


In [4]:
# df_high[df_high[df_high.columns[0]]==' 2'].index[0]

# Data load(우선, 15년도 데이터만)

In [157]:
# df_back[df_back['back_hospital.yyyymmdd']==20150101]

In [8]:
df_back_15 = df_back.iloc[37264:]         # 15년도
df_back_15

,back_hospital.yyyymmdd,back_hospital.area,back_hospital.sex,back_hospital.frequency
37264,20150101,강원,1,4
37265,20150101,강원,2,2
37266,20150101,경기,1,4
37267,20150101,경기,2,6
37268,20150101,경남,1,6
...,...,...,...,...
49669,20151231,제주,2,0
49670,20151231,충남,1,0
49671,20151231,충남,2,0
49672,20151231,충북,1,1


In [9]:
# 한 날짜에 각 지역 별로 남,녀 17개씩 총 34개 분포
submission = pd.read_csv('2-2_검증데이터셋.csv', encoding='cp949')
submission1=submission.copy()
submission1

,yyyymmdd,area,sex,frequency
0,2016-01-01,강원,1,NaN
1,2016-01-01,경기,1,NaN
2,2016-01-01,경남,1,NaN
3,2016-01-01,경북,1,NaN
4,2016-01-01,광주,1,NaN
...,...,...,...,...
12439,2016-12-31,전남,2,NaN
12440,2016-12-31,전북,2,NaN
12441,2016-12-31,제주,2,NaN
12442,2016-12-31,충남,2,NaN


In [10]:
list(submission[submission['yyyymmdd']=='2016-01-01']['area'].values)[:17]        # 이순서대로(채울 땐, 두 겹으로.)

['강원',
 '경기',
 '경남',
 '경북',
 '광주',
 '대구',
 '대전',
 '부산',
 '서울',
 '세종',
 '울산',
 '인천',
 '전남',
 '전북',
 '제주',
 '충남',
 '충북']

# Data Pre-processing

In [11]:
area = list(submission[submission['yyyymmdd']=='2016-01-01']['area'].values)[:17]        # 제출 순서대로
area

['강원',
 '경기',
 '경남',
 '경북',
 '광주',
 '대구',
 '대전',
 '부산',
 '서울',
 '세종',
 '울산',
 '인천',
 '전남',
 '전북',
 '제주',
 '충남',
 '충북']

In [22]:
def preprocess(df):
    df =  pd.get_dummies(df, columns = ['back_hospital.sex'])
    df = df.reset_index()
    df = df.drop(['index', 'back_hospital.yyyymmdd', 'back_hospital.area'], axis=1)
    return df

df_back_li = []     # 지역별 훈련 데이터 프레임 저장
for i in range(len(area)):
    df_back_li.append(preprocess(df_back_15[df_back_15['back_hospital.area'] == area[i]]))


df_back_li[0]

y_freq = []
for k in range(len(df_back_li)):
    for i in range(len(df_back_li[i])):
    y_freq.append(list(df_back_li[i]['back_hospital.frequency'].values))
    
y_freq

IndentationError: expected an indented block (4292551269.py, line 17)

In [20]:
df_back_li[0]['back_hospital.frequency'].values

array([4, 2, 1, 1, 1, 1, 1, 0, 1, 2, 3, 1, 1, 0, 0, 0, 2, 1, 1, 0, 0, 0,
       4, 1, 0, 2, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 2, 3, 1,
       0, 3, 1, 2, 0, 0, 3, 2, 2, 1, 1, 0, 1, 1, 0, 1, 1, 0, 3, 2, 3, 0,
       2, 3, 0, 3, 0, 1, 1, 2, 0, 0, 0, 2, 2, 0, 0, 2, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 0, 2, 2, 2, 1, 0, 0, 3, 2, 2, 0, 3, 0, 1, 1, 1, 1, 1, 2,
       3, 2, 3, 0, 1, 1, 0, 2, 1, 0, 0, 1, 1, 1, 2, 1, 2, 0, 0, 2, 2, 2,
       3, 0, 2, 2, 0, 1, 1, 2, 2, 1, 0, 0, 0, 0, 2, 1, 2, 3, 2, 0, 1, 1,
       2, 4, 1, 1, 4, 1, 3, 1, 2, 1, 0, 0, 1, 1, 0, 0, 2, 2, 0, 0, 1, 3,
       2, 0, 0, 3, 6, 1, 3, 0, 0, 2, 1, 1, 1, 2, 1, 2, 2, 0, 2, 1, 2, 1,
       0, 1, 0, 0, 0, 1, 4, 2, 1, 1, 3, 2, 1, 0, 0, 2, 3, 2, 0, 0, 0, 2,
       1, 1, 1, 0, 2, 1, 2, 0, 0, 0, 1, 0, 6, 0, 0, 0, 1, 0, 1, 0, 3, 1,
       1, 0, 0, 2, 4, 2, 1, 0, 1, 0, 1, 2, 1, 0, 0, 0, 0, 0, 1, 1, 1, 2,
       2, 3, 1, 0, 3, 0, 0, 0, 3, 3, 2, 0, 1, 2, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 2, 0, 2, 1, 1, 0, 3, 3, 4, 3,

In [15]:
def val_preprocess(df):
    df =  pd.get_dummies(df, columns = ['sex'])
    df = df.reset_index()
    df = df.drop(['index', 'yyyymmdd', 'area', 'frequency'], axis=1)
    return df

df_back_val_li = []     # 지역별 검증 데이터 프레임 저장
for i in range(len(area)):
    df_back_val_li.append(val_preprocess(submission1[submission1['area'] == area[i]]))


df_back_val_li[0]

,sex_1,sex_2
0,1,0
1,0,1
2,1,0
3,0,1
4,1,0
...,...,...
727,0,1
728,1,0
729,0,1
730,1,0


# 15년도 외부데이터 열 추가

train

In [4]:
curr_dir = os.getcwd()
curr_dir

'C:\\weather\\back_hospital'

In [5]:
high_path = os.path.join(curr_dir+"\\data\\"+"high")
low_path = os.path.join(curr_dir+"\\data\\"+"low")

In [6]:
high_li = os.listdir(high_path)
high_li

low_li = os.listdir(low_path)
low_li

['강원_갈말읍_일최저기온_201501_201512.csv',
 '경기_문산읍_일최저기온_201501_201512.csv',
 '경남_가포동_일최저기온_201501_201512.csv',
 '경북_남원동_일최저기온_201501_201512.csv',
 '광주_운암1동_일최저기온_201501_201512.csv',
 '대구_효목1동_일최저기온_201501_201512.csv',
 '대전_온천2동_일최저기온_201501_201512.csv',
 '부산_대청동_일최저기온_201501_201512.csv',
 '서울_교남동_일최저기온_201501_201512.csv',
 '세종_새롬동_일최저기온_201501_201512.csv',
 '울산_병영2동_일최저기온_201501_201512.csv',
 '인천_덕적면_일최저기온_201501_201512.csv',
 '전남_강진읍_일최저기온_201501_201512.csv',
 '전북_고창읍_일최저기온_201501_201512.csv',
 '제주_건입동_일최저기온_201501_201512.csv',
 '충남_금산읍_일최저기온_201501_201512.csv',
 '충북_강내면_일최저기온_201501_201512.csv']

In [27]:
# df_back_li[i]['high'] = 0
df_high = pd.read_csv(os.path.join(curr_dir+"\\data\\"+'\\high\\'+high_li[1]))
# df_high = df_high.dropna(subset = ['hour'])

# df_high.iloc[6750:7000]
df_high[df_high[df_high.columns[0]]=='31']


,format: day,hour,forecast,value location:56_133 Start : 20150101


In [40]:
dd = pd.read_csv('dd.csv')
dd[[dd.columns[0]]].values[2][0][12:14]  ##달만 추출   년도/도끼리/읍동들/

'01'

In [46]:
# 최고기온
for i in range(len(high_li)):
    df_back_li[i]['high'] = 0
    df_high = pd.read_csv(os.path.join(curr_dir+"\\data\\"+'\\high\\'+high_li[i]))
    df_high = df_high.dropna(subset = ['hour'])
    
    max = []
    ind = df_high[df_high[df_high.columns[0]]==' 2'].index[0]
    for j in range(len(df_high)//ind):
        max.append(np.max(df_high[df_high.columns[-1]].values[ind*j:ind*(j+1)-1]))
    max_2 = []
    for j in range(len(max)):
        max_2.append(max[j])
        max_2.append(max[j])
    len(max_2)


    for j in range(len(df_back_li[i])-len(max_2)):         # 뒷 부분 맞지 않는 것은 맞춘 끝부분 연장해서
        max_2.append(max_2[-1])
    len(max_2)

    df_back_li[i]['high'] = max_2

In [48]:
# 최저기온
for i in range(len(low_li)):
    df_back_li[i]['low'] = 0
    df_low = pd.read_csv(os.path.join(curr_dir+"\\data\\"+'\\low\\'+low_li[i]))
    df_low = df_low.dropna(subset = ['hour'])
    
    min = []
    ind = df_low[df_low[df_low.columns[0]]==' 2'].index[0]
    for j in range(len(df_low)//ind):
        min.append(np.min(df_low[df_low.columns[-1]].values[ind*j:ind*(j+1)-1]))
    min_2 = []
    for j in range(len(min)):
        min_2.append(min[j])
        min_2.append(min[j])
    len(min_2)


    for j in range(len(df_back_li[i])-len(min_2)):         # 뒷 부분 맞지 않는 것은 맞춘 끝부분 연장해서
        min_2.append(min_2[-1])
    len(min_2)

    df_back_li[i]['low'] = min_2

val

In [88]:
high_val_path = os.path.join(curr_dir+"\\data\\"+"high_val")
low_val_path = os.path.join(curr_dir+"\\data\\"+"low_val")

high_val_li = os.listdir(high_val_path)
high_val_li

low_val_li = os.listdir(low_val_path)
low_val_li

['강원_갈말읍_일최저기온_201601_201612.csv',
 '경기_문산읍_일최저기온_201601_201612.csv',
 '경남_가포동_일최저기온_201601_201612.csv',
 '경북_울릉읍_일최저기온_201601_201612.csv',
 '광주_운암1동_일최저기온_201601_201612.csv',
 '대구_효목1동_일최저기온_201601_201612.csv',
 '대전_온천2동_일최저기온_201601_201612.csv',
 '부산_대청동_일최저기온_201601_201612.csv',
 '서울_교남동_일최저기온_201601_201612.csv',
 '세종_새롬동_일최저기온_201601_201612.csv',
 '울산_병영2동_일최저기온_201601_201612.csv',
 '인천_덕적면_일최저기온_201601_201612.csv',
 '전남_강진읍_일최저기온_201601_201612.csv',
 '전북_고창읍_일최저기온_201601_201612.csv',
 '제주_건입동_일최저기온_201601_201612.csv',
 '충남_금산읍_일최저기온_201601_201612.csv',
 '충북_강내면_일최저기온_201601_201612.csv']

In [91]:
df_back_val_li[0]

,sex_1,sex_2,high
0,1,0,7.0
1,0,1,7.0
2,1,0,8.0
3,0,1,8.0
4,1,0,7.0
...,...,...,...
727,0,1,6.0
728,1,0,6.0
729,0,1,6.0
730,1,0,6.0


In [105]:
len(submission1)/2

6222.0

In [108]:
len(high_val_li[1])

30

In [ ]:
###################달마다 거의 1일치 밖에 없어서 전처리에 무리....

In [111]:
# 최고기온
for i in range(len(high_val_li)):
    df_back_val_li[i]['high'] = 0
    df_val_high = pd.read_csv(os.path.join(curr_dir+"\\data\\"+'\\high_val\\'+high_val_li[i]))
    df_val_high = df_val_high.dropna(subset = ['hour'])
    
    max = []
    ind = df_val_high[df_val_high[df_val_high.columns[0]]==' 2'].index[0]
    for j in range(len(df_val_high)//ind):
        max.append(np.max(df_val_high[df_val_high.columns[-1]].values[ind*j:ind*(j+1)-1]))
    max_2 = []
    for j in range(len(max)):
        max_2.append(max[j])
        max_2.append(max[j])
    len(max_2)


    for j in range(len(df_back_val_li[i])-len(max_2)):         # 뒷 부분 맞지 않는 것은 맞춘 끝부분 연장해서
        max_2.append(max_2[-1])
    len(max_2)

    df_back_val_li[i]['high'] = max_2

ValueError: Length of values (1464) does not match length of index (732)

In [92]:
# 최저기온
for i in range(len(low_val_li)):
    df_back_val_li[i]['low'] = 0
    df_val_low = pd.read_csv(os.path.join(curr_dir+"\\data\\"+'\\low_val\\'+low_val_li[i]))
    df_val_low = df_val_low.dropna(subset = ['hour'])
    
    min = []
    ind = df_val_low[df_val_low[df_val_low.columns[0]]==' 2'].index[0]
    for j in range(len(df_val_low)//ind):
        min.append(np.min(df_val_low[df_val_low.columns[-1]].values[ind*j:ind*(j+1)-1]))
    min_2 = []
    for j in range(len(min)):
        min_2.append(min[j])
        min_2.append(min[j])
    len(min_2)


    for j in range(len(df_back_val_li[i])-len(min_2)):         # 뒷 부분 맞지 않는 것은 맞춘 끝부분 연장해서
        min_2.append(min_2[-1])
    len(min_2)

    df_back_val_li[i]['low'] = min_2

ValueError: Length of values (1464) does not match length of index (732)

In [94]:
submission1[submission1['area']=='강원']

,yyyymmdd,area,sex,frequency
0,2016-01-01,강원,1,NaN
17,2016-01-01,강원,2,NaN
34,2016-01-02,강원,1,NaN
51,2016-01-02,강원,2,NaN
68,2016-01-03,강원,1,NaN
...,...,...,...,...
12359,2016-12-29,강원,2,NaN
12376,2016-12-30,강원,1,NaN
12393,2016-12-30,강원,2,NaN
12410,2016-12-31,강원,1,NaN


In [75]:
# # 최저기온
# for i in range(len(low_val_li)):
#     submission1['low'] = 0
#     df_val_low = pd.read_csv(os.path.join(curr_dir+"\\data\\"+'\\low_val\\'+low_val_li[i]))
#     df_val_low = df_val_low.dropna(subset = ['hour'])
    
#     min = []
#     ind = df_val_low[df_val_low[df_val_low.columns[0]]==' 2'].index[0]
#     for j in range(len(df_val_low)//ind):
#         min.append(np.min(df_val_low[df_val_low.columns[-1]].values[ind*j:ind*(j+1)-1]))
#     min_2 = []
#     for j in range(len(min)):
#         min_2.append(min[j])
#         min_2.append(min[j])
#     len(min_2)


#     for j in range(len(submission1)-len(min_2)):         # 뒷 부분 맞지 않는 것은 맞춘 끝부분 연장해서
#         min_2.append(min_2[-1])
#     len(min_2)

#     submission1['low'] = min_2

In [76]:
submission1

,yyyymmdd,area,sex,frequency,high,low
0,2016-01-01,강원,1,NaN,9.900000,-0.100000
1,2016-01-01,경기,1,NaN,9.900000,-0.100000
2,2016-01-01,경남,1,NaN,8.000000,-2.700000
3,2016-01-01,경북,1,NaN,8.000000,-2.700000
4,2016-01-01,광주,1,NaN,8.100000,-1.900000
...,...,...,...,...,...,...
12439,2016-12-31,전남,2,NaN,9.400000,-1.300000
12440,2016-12-31,전북,2,NaN,9.400000,-1.300000
12441,2016-12-31,제주,2,NaN,9.400000,-1.300000
12442,2016-12-31,충남,2,NaN,9.400000,-1.300000


# Train & Submission

In [ ]:
# 환절기에 접어드는 10월과 1,2월은 혈관수축으로 인한 뇌혈관질환자가 특히 많이 발생합니다. 

# 그 날짜의 (단기예보)각 지역 별 최저,최고 기온의 평균-> 그 지역 기온(가장 높은/낮은 것만)으로 하고, 피쳐 생성
# 우선, 각 지역의 대표 하나(우선, 맨 위에것) 설정한 '동/읍/면/리'에서 최저,최고 이를 이용.

#----------------------------------
# 추후, 지역 뿐아니라 성별을 가를 만한 자료가 있으면 좋을 듯 => 없다면, dummy변수로.
# 시계열로 가면 좋을지
# train에서 train/val 나누고, sub을 test써서 => 데이터 수가 적어서 이를 나누어도 될지
# 계절 별로 나누어야 할까? => '기온'과 같은 피쳐를 사용한다면, 이를 대신하기 때문에 괜찮을듯함

In [ ]:
max_depth = 5
n_estimators = 5000             # 400
lr = 0.01
num_leaves = 31
early_stopping_rounds = 300      # 100
verbose = 500                  # True

In [ ]:
def model_train(curr_dir):
 
    path_train = os.path.join(curr_dir+"\\data_all\\"+"\\train")
    path_test = os.path.join(curr_dir+"\\data_all\\"+"\\test")
    
    folders_tr = os.listdir(path_train)
    folders_test = os.listdir(path_test)
    
    train = pd.DataFrame()
    for files in folders_tr:
        df= pd.read_csv(os.path.join(curr_dir+"\\data_all\\"+"\\train\\")+files)
        train = pd.concat([train, df])            
              
    
    test = pd.DataFrame()
    for files in folders_test:
        df= pd.read_csv(os.path.join(curr_dir+"\\data_all\\"+"\\test\\")+files)
        test = pd.concat([test, df])   
         

        
        
    X_train = train[['Lat','SolarZA','SateZA','ESR','Band1','Band2','Band3','Band4','Band6','Band7', '30daysBand3','30daysBand13',\
                     'GK2A-LST','insitu-TD','insitu-TG', 'insitu-HM','insitu-PS','insitu-TED0.05','insitu-TED0.1','insitu-TED0.2',\
                     'insitu-TED0.3','insitu-TED0.5','insitu-TED1.0','insitu-TED1.5','insitu-TED5.0']]
    X_test = test[['Lat','SolarZA','SateZA','ESR','Band1','Band2','Band3','Band4','Band6','Band7', '30daysBand3','30daysBand13',\
                     'GK2A-LST','insitu-TD','insitu-TG', 'insitu-HM','insitu-PS','insitu-TED0.05','insitu-TED0.1','insitu-TED0.2',\
                     'insitu-TED0.3','insitu-TED0.5','insitu-TED1.0','insitu-TED1.5','insitu-TED5.0']]
    y_train_LST=train[['isitu-LST']].values
    y_test_LST=test[['isitu-LST']].values
    
    y_train_TA=train[['insitu-TA']].values
    y_test_TA=test[['insitu-TA']].values
    
#     model_name = '\\save\\'+'best_model_v1.h5' 

#     filepath = os.path.join(os.getcwd()+ model_name)

#     es = EarlyStopping(monitor='val_mse', verbose=1, patience=20)

#     checkpoint = ModelCheckpoint(filepath=filepath,
#                              monitor='val_mse',
#                              verbose=1,
#                              save_best_only=True)

#     callbacks = [checkpoint, es]
    xgb_model = xgb.XGBRegressor(n_estimators = 2500, learning_rate = 0.017, max_depth=11, subsample=0.5,colsample_bytree=0.7, \
                                   random_state = 2000).fit(X_train,y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=15)
    
#     lgbm_reg.load_weights(model_name)
    
    
    path_val = os.path.join(curr_dir+"\\data_all\\"+"\\validation")
    folders_val = os.listdir(path_val)
    
    validation = pd.DataFrame()
    for files in folders_val:
        df= pd.read_csv(os.path.join(curr_dir+"\\data_all\\"+"\\validation\\")+files)
        validation = pd.concat([validation, df])        
        
    validation_ = validation[['Lat','SolarZA','SateZA','ESR','Band1','Band2','Band3','Band4','Band6','Band7', '30daysBand3','30daysBand13',\
                     'GK2A-LST','insitu-TD','insitu-TG', 'insitu-HM','insitu-PS','insitu-TED0.05','insitu-TED0.1','insitu-TED0.2',\
                     'insitu-TED0.3','insitu-TED0.5','insitu-TED1.0','insitu-TED1.5','insitu-TED5.0']]
    prediction_LST = lgbm_reg_LST.predict(validation_)
    prediction_TA = lgbm_reg_TA.predict(validation_)
    
    validation_partial = validation[['YearMonthDayHourMinute', 'STN']]
    validation_partial = validation_partial.assign(LST = prediction_LST)
    validation_partial = validation_partial.assign(TA = prediction_TA)
    
    submission1[['isitu-LST', 'insitu-TA']] = validation_partial[['LST','TA']].values
    submission1 = submission1.drop(['high','low'], axis=1)
    submission1.to_csv("lgbm_v1.csv", index = False)

In [ ]:
model_train(curr_dir)